**将问题框架化并关注重点**

1. 用业务术语定义目标
    * 预测给定信息用户的还款能力，以概率的方式
2. 你的解决方案将如何使用？
    * 用来预测未来贷款申请客户的好坏，作为申请决策辅助依据。
3. 目前的解决方案/解决方法（如果有的话）是什么？
    * 未知
4. 你应该如何解决这个问题（监督/非监督，在线/离线等）？
    * 使用监督学习，回归问题，label中‘0’表示好人，‘1’表示坏人，给出的数介于两者之间，清洗好特征后，多模型测试
5. 如何度量模型的表现？
    * 使用roc下的面积
6. 模型的表现是否和业务目标一致？
    * 纯模型训练，无实际业务目标，暂且认为一致
7. 达到业务目标所需的最低性能是多少？
    * 未知，一般任务auc=0.75
8. 类似的问题如何解决？是否可以复用经验或工具？
    * pass
9. 人员是否专业？
    * pass
10. 你如何动手解决问题？
    * pass
11. 列出目前你（或者其他人）所做的假设
    * pass
12. 如果可能，验证假设。

---

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

**获取数据**

注意：尽可能自动化，以便你轻松获取新数据。

1. 列出你需要的数据和数据量。
    * 已给定
2. 查找并记录你可以获取该数据的位置。
    * pass
3. 检查它将占用多少存储空间。
    * 2.5G
4. 检查法律义务并在必要时获取授权。
    *  开放数据集，无需授权
5. 获取访问权限。
    * kaggle
6. 创建工作目录（拥有足够的存储空间）。
    * pass
7. 获取数据。
    * 如下
8. 将数据转换为你可以轻松操作的格式（不更改数据本身）。
    * pass
9. 确保删除或保护敏感信息（比如，匿名）。
    * 数据集本身已处理
10. 检查数据的大小和类型（时间序列，样本，地理信息等）。
    * 如下
11. 抽样出测试集，将它放在一边，以后不需要关注它（没有数据窥探！）。
    * 如下

---

In [2]:
# 使用kaggle命令获取数据并解压
# &&kaggle competitions download -c home-credit-default-risk\
# &&mkdir data&&unzip *.zip -d ./data/

In [3]:
# 查看数据占用空间 
!du -sh ./data/

2.5G	./data/


In [4]:
def load_data(data_set_name:str):
    return pd.read_csv('./data/{}.csv'.format(data_set_name))

In [5]:
application = load_data('application_train')
# application_test = load_data('application_test')
application.shape
# application.columns.difference(application_test.columns)
# application_test.shape
# del application_test

(307511, 122)

In [6]:
def reduce_mem_usage(df, ignore_cols=['SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV']):
    """在不损失数据信息的情况下，通过转换数字数据类型来减少数据帧的内存使用"""
    # 初始化数据框的内存使用
    start_mem = df.memory_usage().sum() / 1024**2
    print('初始内存使用: {:.2f} MB'.format(start_mem))
    # 剔除特定列
    cols = [ col for col in df.columns if col not in ignore_cols]
    # 遍历每一列
    for col in cols:
        col_type = df[col].dtype
        
        # 如果数据类型是整数类型
        if col_type != object and col_type.name != 'category' and 'datetime' not in col_type.name:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # 如果最小值和最大值都可以用更小的数据类型表示
            if str(col_type)[:3] == 'int' and c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif str(col_type)[:3] == 'int' and c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif str(col_type)[:3] == 'int' and c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            elif str(col_type)[:3] == 'int' and c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                df[col] = df[col].astype(np.int64)
            
            # 如果最小值和最大值都可以用更小的数据类型表示
            elif str(col_type)[:5] == 'float' and c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif str(col_type)[:5] == 'float' and c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                pass

    # 输出优化后的内存使用
    end_mem = df.memory_usage().sum() / 1024**2
    print('优化后的内存使用: {:.2f} MB'.format(end_mem))
    return df

In [7]:
application = reduce_mem_usage(application)

初始内存使用: 286.23 MB
优化后的内存使用: 93.55 MB


**快速看一眼数据**

In [8]:
application.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float16(61), float32(4), int16(2), int32(1), int64(1), int8(37), object(16)
memory usage: 93.6+ MB


**创建测试集**

因为kaggle比赛的application_test没有target，无法测量ROC下的面积，所以丢弃不用，切分application_train为训练集和测试集

在切分数据集的时候有多种策略可以选择，一般而言，可以随机切分 也可以分层切分，在本场景下，数据集比较大，坏样本相对较多，所以随机切；

**随机切分**

In [9]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(application, test_size=0.2, random_state= 42)

**探索数据**

注意：尝试从领域专家那获取有关这些步骤的见解。

1. 创建用于探索的数据副本（如有必要，将其取样为可管理的大小）。
2. 创建一个 Jupyter 笔记本来记录你的数据探索。
3. 研究每个属性及其特征：
  * 名称；
  * 类型（分类，整数/浮点数，有界/无界，文本，结构化数据等）；
  * 缺失数据的百分比；
  * 噪声点和它的类型（随机点，异常点，舍入误差等）；
  * 对任务可能有用吗？
  * 分布类型（高斯分布，均匀分布，对数分布等）。
4. 对于监督学习任务，确定目标属性。
5. 可视化数据。
6. 研究属性间的相关性。
7. 研究怎如何手动解决问题。
8. 确定你想要应用的有效的转换。
9. 确定有用的额外数据（回到上一步）。
10. 记录你所学到的知识。


In [10]:
app = train_set.copy()

In [11]:
app.dtypes.value_counts()

float16    61
int8       37
object     16
float32     4
int16       2
int64       1
int32       1
dtype: int64

In [12]:
def missing_values_summary(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100*df.isnull().sum() / len(df)
    mis_val_table =pd.concat([mis_val, mis_val_percent], axis=1)   
    mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] !=0].sort_values(
    '% of Total Values', ascending = False).round(1)
    mis_val_table_ren_columns = mis_val_table_ren_columns.merge(df.dtypes.rename('dtype').to_frame(),left_index=True, right_index=True)
    print('Your selected dataframe has ' + str(df.shape[1])+ ' columns.\n'
         "There are " + str(mis_val_table_ren_columns.shape[0])+ ' columns that have missing values.')
    return mis_val_table_ren_columns

In [13]:
missing_values_summary(app)

Your selected dataframe has 122 columns.
There are 67 columns that have missing values.


,Missing Values,% of Total Values,dtype
COMMONAREA_MEDI,171929,69.9,float16
COMMONAREA_AVG,171929,69.9,float16
COMMONAREA_MODE,171929,69.9,float16
NONLIVINGAPARTMENTS_MEDI,170868,69.5,float16
NONLIVINGAPARTMENTS_MODE,170868,69.5,float16
NONLIVINGAPARTMENTS_AVG,170868,69.5,float16
FONDKAPREMONT_MODE,168286,68.4,object
LIVINGAPARTMENTS_MODE,168196,68.4,float16
LIVINGAPARTMENTS_MEDI,168196,68.4,float16
LIVINGAPARTMENTS_AVG,168196,68.4,float16


-----

**简单筛选一下特征**

In [14]:
# 筛选特征
cate_cols = []
num_cols = []

In [15]:
# 将类别属性放进去
cate_cols.extend(app.dtypes[app.dtypes == 'object'].index.tolist())

In [16]:
def select_low_cardinality_numeric_features(df, label_col, threshold=5):
    """
    挑选出除了 label_col 以外的数值型特征，并判断其唯一值数是否低于 threshold，
    如果是，则将该特征的名称加入到列表 low_cardinality_feats 中返回。

    Parameters:
    ----------
    df: pandas.DataFrame
        数据表，包含了特征和目标变量
    label_col: str
        目标变量的名称
    threshold: int
        指定唯一值数量的阈值，低于该值的特征将被视为“唯一值较少的特征”，默认值为 5。

    Returns:
    ----------
    low_cardinality_feats: list
        唯一值较少的数值型特征的名称列表
    """
    numeric_feats = df.select_dtypes(include='number').columns.tolist()
    low_cardinality_feats = []
    for feat in numeric_feats:
        if feat == label_col:
            continue
        if df[feat].nunique() <= threshold:
            low_cardinality_feats.append(feat)
    return low_cardinality_feats


In [17]:
# 将唯一值少于5个的数值型变量也放进去
cate_cols.extend(select_low_cardinality_numeric_features(app, 'TARGET'))

In [18]:
num_cols.extend(app.columns.difference(cate_cols))

In [19]:
num_cols.remove('TARGET')

In [20]:
cat_cols_object = app[cate_cols].select_dtypes(include=['object']).columns
cat_cols_number = app[cate_cols].select_dtypes(include=['number']).columns

---

**准备数据**

注意：
* 处理数据副本（保持原始数据集完整）。
* 为你应用的所有数据转换编写函数，原因有五：
    * 你可以在下次获得新数据集时轻松准备数据
    * 你可以在未来的项目中应用这些转换
    * 用来清洗和准备测试数据集
    * 一旦项目上线你可以用来清洗和准备新的数据集
    * 为了便于将你的准备选择视为超参数

1. 数据清洗：
    * 修正或移除异常值（可选）。
    * 填补缺失值（比如用零，平均值，中位数等）或者删除所在行（或者列）。
2. 特征提取（可选）：
    * 丢弃不提供有用信息的属性；
3. 适当的特征工程：
    * 连续特征离散化。
    * 分解特征（比如分类，日期/时间等）。
    * 对特征添加有益的转换（比如 log(x)，sqrt(x)，x^2 等）
    * 将一些特征融合为有益的新特征
4. 特征缩放：
    * 标准化或者正规化特征。

**准备数据副本**

In [21]:
app = train_set.drop('TARGET', axis=1)
app_labels = train_set['TARGET'].copy()

In [22]:
app.shape

(246008, 121)

**数据清洗**

1. 上面已经列清楚了哪些列有缺失值，分析具体情况进行填充
2. 针对数值型的列，使用具体情况具体分析进行填充，一般来说，数据服从正态分布，使用均值；不服从使用中位数，如果是离散的数据点，可以使用众数；如果缺失较少，使用均值或者中位数，可以保证数据整体分布，缺失较多，可以使用众数，分布更加稳定
3. 针对对象型，使用特殊标记‘UKN’进行填充

In [23]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

In [24]:
import sklearn
sklearn.set_config(display="diagram")

In [25]:
num_pipeline = make_pipeline(
                SimpleImputer(strategy='median'),
                StandardScaler()
                )

In [26]:
cat_number_pipeline = make_pipeline(
                FunctionTransformer(lambda X: X.astype(str),feature_names_out='one-to-one'),
                SimpleImputer(strategy='constant', fill_value='UKN'),
                OneHotEncoder(handle_unknown='ignore')
)

In [27]:
cat_object_pipeline = make_pipeline(
                SimpleImputer(strategy='constant', fill_value='UKN'),
                OneHotEncoder(handle_unknown='ignore')
)

In [28]:
processing = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cate_object',cat_object_pipeline,cat_cols_object),
    ('cate_number', cat_number_pipeline, cat_cols_number)
], remainder='passthrough')

In [29]:
app_transform = processing.fit_transform(app)

In [30]:
app_transform.shape

(246008, 291)

In [31]:
app_transformed = pd.DataFrame(app_transform, columns=processing.get_feature_names_out())

**列出有用模型**

注意：

  * 如果数据量巨大，你可能需要采样出较小的训练集，以便在合理的时间内训练许多不同的模型（请注意，这会对诸如大型神经网络或随机森林等复杂模型进行处罚）。
  * 再次尝试尽可能自动化这些步骤。

1. 使用标准参数训练许多快速、粗糙的模型（比如线性模型，朴素贝叶斯模型，支持向量机模型，随机森林模型，神经网络等）。
2. 衡量并比较他们的表现。
  * 对于每个模型，使用 N 折交叉验证法，并且计算基于 N 折交叉验证的均值与方差。
3. 分析每种算法的最重要变量。
4. 分析模型产生的错误类型。
  * 人们用什么数据来避免这些错误？
5. 进行一轮快速的特征提取和特征工程。
6. 对之前的五个步骤进行一两次的快速迭代。
7. 列出前三到五名最有用的模型，由其是产生不同类型错误的模型。


In [32]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import time


def cross_validate_with_feature_importance(models, data, labels, n_folds=5):
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    for model_idx, model in enumerate(models):
        print(f"Model {model_idx + 1}: {type(model).__name__}")
        total_feature_importance = None
        
        for fold, (train_idx, val_idx) in enumerate(skf.split(data, labels)):
            print(f"Fold {fold + 1}")
            X_train, y_train = data.iloc[train_idx], labels.iloc[train_idx].values
            X_val, y_val = data.iloc[val_idx], labels.iloc[val_idx].values

            start_time = time.time()
            model.fit(X_train, y_train)
            train_time = time.time() - start_time
            print(f"Training time: {train_time:.2f}s")

            if fold == 0 and train_time > 180:
                print(f"Model training skipped because it took {train_time:.2f}s")
                break

            if hasattr(model, "predict_proba"):
                y_pred = model.predict_proba(X_val)[:, 1]
            else:
                y_pred = model.predict(X_val)
            score = roc_auc_score(y_val, y_pred)
            print(f"Validation ROC AUC score: {score:.4f}")

            if hasattr(model, "feature_importances_"):
                fold_feature_importance = pd.Series(model.feature_importances_, index=data.columns)
                if total_feature_importance is None:
                    total_feature_importance = fold_feature_importance
                else:
                    total_feature_importance += fold_feature_importance

        if total_feature_importance is not None:
            top_features = total_feature_importance.sort_values(ascending=False)[:10]
            print("Top 10 most important features:")
            print(top_features)


## Baseline

In [33]:
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPRegressor

lr = LinearRegression()
sgd = SGDRegressor()
tree = DecisionTreeRegressor()
svr = SVR()
# gpr = GaussianProcessRegressor()
gnb = GaussianNB()
nn = MLPRegressor()

In [34]:
models = [lr, sgd, tree, gnb, nn]

In [35]:
# cross_validate_with_feature_importance(models, app_transformed, app_labels)

In [36]:
del app,app_transform,app_transformed,app_labels

先粗略的跑一下模型，不做任何处理，LR和NN可以取得0.75的成绩，还不错

## add more information

In [37]:
import pandas as pd
import gc

def concat_df_by_name(name_str):
    all_vars = globals()
    df_list = []
    keys_to_delete = []  # 存储需要删除的键值对
    for var_name, var_value in all_vars.items():
        if isinstance(var_value, pd.DataFrame) and name_str in var_name:
            df_list.append(var_value)
            keys_to_delete.append(var_name)  # 记录需要删除的键
    for key in keys_to_delete:  # 在循环结束后删除键值对
        del all_vars[key]
    if not df_list:
        return pd.DataFrame()
    result = pd.concat(df_list, axis=1)
    gc.collect()
    return result

In [38]:
bureau = load_data('bureau')
bureau = reduce_mem_usage(bureau)
bureau.CREDIT_TYPE.value_counts()

初始内存使用: 222.62 MB
优化后的内存使用: 126.04 MB


Consumer credit                                 1251615
Credit card                                      402195
Car loan                                          27690
Mortgage                                          18391
Microloan                                         12413
Loan for business development                      1975
Another type of loan                               1017
Unknown type of loan                                555
Loan for working capital replenishment              469
Cash loan (non-earmarked)                            56
Real estate loan                                     27
Loan for the purchase of equipment                   19
Loan for purchase of shares (margin lending)          4
Mobile operator loan                                  1
Interbank credit                                      1
Name: CREDIT_TYPE, dtype: int64

In [39]:
# bureau表
bureau = load_data('bureau')
bureau = reduce_mem_usage(bureau)

# 征信返回总笔数
BUREAU_NUM = bureau[['SK_ID_CURR','SK_ID_BUREAU']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_NUM'})
# 征信报告中活跃贷款笔数
BUREAU_ACTIVE_NUM = bureau[bureau['CREDIT_ACTIVE']=='Active'][['SK_ID_CURR','SK_ID_BUREAU']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_ACTIVE_NUM'})
# 征信报告中关闭贷款笔数
BUREAU_COLSED_NUM = bureau[bureau['CREDIT_ACTIVE']=='Closed'][['SK_ID_CURR','SK_ID_BUREAU']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_Closed_NUM'})
# 征信报告中出售贷款笔数
BUREAU_SOLD_NUM = bureau[bureau['CREDIT_ACTIVE']=='Sold'][['SK_ID_CURR','SK_ID_BUREAU']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_SOLD_NUM'})
# 征信报告中坏账贷款笔数
BUREAU_BAD_DEBT_NUM = bureau[bureau['CREDIT_ACTIVE']=='Bad debt'][['SK_ID_CURR','SK_ID_BUREAU']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_BAD_DEBT_NUM'})
# 最早一次贷款距今时长
BUREAU_MINDAY_APPLICATION = bureau[['SK_ID_CURR','DAYS_CREDIT']].groupby(['SK_ID_CURR']).min().rename(columns={'DAYS_CREDIT':'BUREAU_MINDAY_APPLICATION'})
# 最近一次贷款距今时长
BUREAU_MAXDAY_APPLICATION = bureau[['SK_ID_CURR','DAYS_CREDIT']].groupby(['SK_ID_CURR']).max().rename(columns={'DAYS_CREDIT':'BUREAU_MAXDAY_APPLICATION'})
# 历史上有过逾期的贷款笔数
BUREAU_NUM_OVERDUE = bureau[bureau['CREDIT_DAY_OVERDUE'] > 0][['SK_ID_CURR','CREDIT_DAY_OVERDUE']].groupby(['SK_ID_CURR']).count().rename(columns={'CREDIT_DAY_OVERDUE':'BUREAU_NUM_OVERDUE'})
# 历史上最长逾期天数
BUREAU_MAXDAY_OVERDUE = bureau[bureau['CREDIT_DAY_OVERDUE'] > 0][['SK_ID_CURR','CREDIT_DAY_OVERDUE']].groupby(['SK_ID_CURR']).max().rename(columns={'CREDIT_DAY_OVERDUE':'BUREAU_MAXDAY_OVERDUE'})
# 提前还款的笔数
BUREAU_NUM_PREPAY = bureau[bureau['DAYS_CREDIT_ENDDATE'] > bureau['DAYS_ENDDATE_FACT'] ][['SK_ID_CURR','DAYS_ENDDATE_FACT']].groupby(['SK_ID_CURR']).count().rename(columns={'DAYS_ENDDATE_FACT':'BUREAU_NUM_PREPAY'})
# 到期还款笔数
BUREAU_NUM_NORMAL = bureau[bureau['DAYS_CREDIT_ENDDATE'] == bureau['DAYS_ENDDATE_FACT'] ][['SK_ID_CURR','DAYS_ENDDATE_FACT']].groupby(['SK_ID_CURR']).count().rename(columns={'DAYS_ENDDATE_FACT':'BUREAU_NUM_NORMAL'})
# 延后还款笔数
BUREAU_NUM_DELAY = bureau[bureau['DAYS_CREDIT_ENDDATE'] < bureau['DAYS_ENDDATE_FACT'] ][['SK_ID_CURR','DAYS_ENDDATE_FACT']].groupby(['SK_ID_CURR']).count().rename(columns={'DAYS_ENDDATE_FACT':'BUREAU_NUM_DELAY'})
# 历史最大逾期金额
BUREAU_MAXAMT_OVERDUE = bureau[bureau['AMT_CREDIT_MAX_OVERDUE'] > 0][['SK_ID_CURR','AMT_CREDIT_MAX_OVERDUE']].groupby(['SK_ID_CURR']).max().rename(columns={'AMT_CREDIT_MAX_OVERDUE':'BUREAU_MAXAMT_OVERDUE'})
# 总展期次数
BUREAU_PROLONG_NUM = bureau[['SK_ID_CURR','CNT_CREDIT_PROLONG']].groupby(['SK_ID_CURR']).max().rename(columns={'CNT_CREDIT_PROLONG':'BUREAU_PROLONG_NUM'})
# 总还款金额
BUREAU_LOAN_AMT = bureau[['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM':'BUREAU_LOAN_AMT'})
# 总未还金额
BUREAU_DEBT_AMT = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_DEBT':'BUREAU_DEBT_AMT'})
# 最大信用额度
BUREAU_LIMIT_MAX = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_LIMIT']].groupby(['SK_ID_CURR']).max().rename(columns={'AMT_CREDIT_SUM_LIMIT':'BUREAU_LIMIT_MAX'})
# 总逾期金额
BUREAU_SUM_OVERDUE = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_OVERDUE':'BUREAU_SUM_OVERDUE'})
# 消费贷数
BUREAU_NUM_CONSUMER = bureau[bureau['CREDIT_TYPE'] == 'Consumer credit' ][['SK_ID_CURR','CREDIT_TYPE']].groupby(['SK_ID_CURR']).count().rename(columns={'CREDIT_TYPE':'BUREAU_NUM_CONSUMER'})
# 信用卡数
BUREAU_NUM_CARD = bureau[bureau['CREDIT_TYPE'] == 'Credit card' ][['SK_ID_CURR','CREDIT_TYPE']].groupby(['SK_ID_CURR']).count().rename(columns={'CREDIT_TYPE':'BUREAU_NUM_CARD'})
# 汽车贷款数
BUREAU_NUM_CAR = bureau[bureau['CREDIT_TYPE'] == 'Car loan' ][['SK_ID_CURR','CREDIT_TYPE']].groupby(['SK_ID_CURR']).count().rename(columns={'CREDIT_TYPE':'BUREAU_NUM_CAR'})
# 抵押贷款数
BUREAU_NUM_MORTGAGE = bureau[bureau['CREDIT_TYPE'] == 'Mortgage' ][['SK_ID_CURR','CREDIT_TYPE']].groupby(['SK_ID_CURR']).count().rename(columns={'CREDIT_TYPE':'BUREAU_NUM_MORTGAGE'})
# 小微贷款数
BUREAU_NUM_MiCROLOAN = bureau[bureau['CREDIT_TYPE'] == 'Microloan' ][['SK_ID_CURR','CREDIT_TYPE']].groupby(['SK_ID_CURR']).count().rename(columns={'CREDIT_TYPE':'BUREAU_NUM_MiCROLOAN'})
# 其他贷款数
BUREAU_NUM_OTHER = bureau[~bureau.CREDIT_TYPE.isin(['Consumer credit','Credit card','Car loan','Mortgage','Microloan']) ][['SK_ID_CURR','CREDIT_TYPE']].groupby(['SK_ID_CURR']).count().rename(columns={'CREDIT_TYPE':'BUREAU_NUM_OTHER'})
# 消费贷借款总金额
BUREAU_AMT_CONSUMER = bureau[bureau['CREDIT_TYPE'] == 'Consumer credit' ][['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM':'BUREAU_AMT_CONSUMER'})
# 信用卡借款总金额
BUREAU_AMT_CARD = bureau[bureau['CREDIT_TYPE'] == 'Credit card' ][['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM':'BUREAU_AMT_CARD'})
# 汽车贷款借款总金额
BUREAU_AMT_CAR = bureau[bureau['CREDIT_TYPE'] == 'Car loan' ][['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM':'BUREAU_AMT_CAR'})
# 抵押贷款借款总金额
BUREAU_AMT_MORTGAGE = bureau[bureau['CREDIT_TYPE'] == 'Mortgage' ][['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM':'BUREAU_AMT_MORTGAGE'})
# 小微贷款借款总金额
BUREAU_AMT_MiCROLOAN = bureau[bureau['CREDIT_TYPE'] == 'Microloan' ][['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM':'BUREAU_AMT_MiCROLOAN'})
# 其他贷款借款总金额
BUREAU_AMT_OTHER = bureau[~bureau.CREDIT_TYPE.isin(['Consumer credit','Credit card','Car loan','Mortgage','Microloan']) ][['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM':'BUREAU_AMT_OTHER'})
# 消费贷借款未还总金额
BUREAU_DEBTAMT_CONSUMER = bureau[bureau['CREDIT_TYPE'] == 'Consumer credit' ][['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_DEBT':'BUREAU_DEBTAMT_CONSUMER'})
# 信用卡借款未还总金额
BUREAU_DEBTAMT_CARD = bureau[bureau['CREDIT_TYPE'] == 'Credit card' ][['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_DEBT':'BUREAU_DEBTAMT_CARD'})
# 汽车贷款借款未还总金额
BUREAU_DEBTAMT_CAR = bureau[bureau['CREDIT_TYPE'] == 'Car loan' ][['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_DEBT':'BUREAU_DEBTAMT_CAR'})
# 抵押贷款借款未还总金额
BUREAU_DEBTAMT_MORTGAGE = bureau[bureau['CREDIT_TYPE'] == 'Mortgage' ][['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_DEBT':'BUREAU_DEBTAMT_MORTGAGE'})
# 小微贷款借款未还总金额
BUREAU_DEBTAMT_MiCROLOAN = bureau[bureau['CREDIT_TYPE'] == 'Microloan' ][['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_DEBT':'BUREAU_DEBTAMT_MiCROLOAN'})
# 其他贷款借款未还总金额
BUREAU_DEBTAMT_OTHER = bureau[~bureau.CREDIT_TYPE.isin(['Consumer credit','Credit card','Car loan','Mortgage','Microloan']) ][['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_DEBT':'BUREAU_DEBTAMT_OTHER'})
# 消费贷逾期总金额
BUREAU_OVERDUEAMT_CONSUMER = bureau[bureau['CREDIT_TYPE'] == 'Consumer credit' ][['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_OVERDUE':'BUREAU_OVERDUEAMT_CONSUMER'})
# 信用卡逾期总金额
BUREAU_OVERDUEAMT_CARD = bureau[bureau['CREDIT_TYPE'] == 'Credit card' ][['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_OVERDUE':'BUREAU_OVERDUEAMT_CARD'})
# 汽车贷款逾期总金额
BUREAU_OVERDUEAMT_CAR = bureau[bureau['CREDIT_TYPE'] == 'Car loan' ][['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_OVERDUE':'BUREAU_OVERDUEAMT_CAR'})
# 抵押贷款逾期总金额
BUREAU_OVERDUEAMT_MORTGAGE = bureau[bureau['CREDIT_TYPE'] == 'Mortgage' ][['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_OVERDUE':'BUREAU_OVERDUEAMT_MORTGAGE'})
# 小微贷款逾期总金额
BUREAU_OVERDUEAMT_MiCROLOAN = bureau[bureau['CREDIT_TYPE'] == 'Microloan' ][['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_OVERDUE':'BUREAU_OVERDUEAMT_MiCROLOAN'})
# 其他贷款逾期总金额
BUREAU_OVERDUEAMT_OTHER = bureau[~bureau.CREDIT_TYPE.isin(['Consumer credit','Credit card','Car loan','Mortgage','Microloan']) ][['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_OVERDUE':'BUREAU_OVERDUEAMT_OTHER'})
# 最近一次更新记录时间
BUREAU_LAST_UPDATE = bureau[['SK_ID_CURR','DAYS_CREDIT_UPDATE']].groupby(['SK_ID_CURR']).max().rename(columns={'DAYS_CREDIT_UPDATE':'BUREAU_LAST_UPDATE'})


初始内存使用: 222.62 MB
优化后的内存使用: 126.04 MB


In [40]:
bureau_extra = concat_df_by_name('BUREAU')
bureau_extra.shape

(305811, 43)

In [41]:
# bureau_balance表
bureau_balance = load_data('bureau_balance')
bureau_balance = reduce_mem_usage(bureau_balance)

tmp = bureau[['SK_ID_BUREAU','SK_ID_CURR']]
bureau_balance_union = pd.merge(bureau_balance, tmp, how='left', on='SK_ID_BUREAU')
bureau_balance_union['SK_ID_CURR'] = bureau_balance_union['SK_ID_CURR'].fillna(0).astype('int64')
bureau_balance_union = bureau_balance_union[bureau_balance_union.STATUS.isin(['0','1','2','3','4','5'])]

# 征信最近1个月逾期状态 0 1 2 3 4 5的笔数
tmp = bureau_balance_union[bureau_balance_union.MONTHS_BALANCE >= -1][['SK_ID_BUREAU','SK_ID_CURR','STATUS']].groupby(['SK_ID_BUREAU','SK_ID_CURR'], as_index=False).max()
BUREAU_OVERDUE_1_0 = tmp[tmp.STATUS =='0'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_1_0'})
BUREAU_OVERDUE_1_1 = tmp[tmp.STATUS == '1'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_1_1'})
BUREAU_OVERDUE_1_2 = tmp[tmp.STATUS == '2'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_1_2'})
BUREAU_OVERDUE_1_3 = tmp[tmp.STATUS == '3'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_1_3'})
BUREAU_OVERDUE_1_4 = tmp[tmp.STATUS == '4'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_1_4'})
BUREAU_OVERDUE_1_5 = tmp[tmp.STATUS == '5'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_1_5'})

# 征信最近1-3个月逾期状态 0 1 2 3 4 5的笔数
tmp = bureau_balance_union[ (bureau_balance_union.MONTHS_BALANCE < -1) & (bureau_balance_union.MONTHS_BALANCE >= -3) ][['SK_ID_BUREAU','SK_ID_CURR','STATUS']].groupby(['SK_ID_BUREAU','SK_ID_CURR'], as_index=False).max()
BUREAU_OVERDUE_3_0 = tmp[tmp.STATUS =='0'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_3_0'})
BUREAU_OVERDUE_3_1 = tmp[tmp.STATUS == '1'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_3_1'})
BUREAU_OVERDUE_3_2 = tmp[tmp.STATUS == '2'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_3_2'})
BUREAU_OVERDUE_3_3 = tmp[tmp.STATUS == '3'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_3_3'})
BUREAU_OVERDUE_3_4 = tmp[tmp.STATUS == '4'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_3_4'})
BUREAU_OVERDUE_3_5 = tmp[tmp.STATUS == '5'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_3_5'})

# 征信最近3-6个月逾期状态 0 1 2 3 4 5的笔数
tmp = bureau_balance_union[ (bureau_balance_union.MONTHS_BALANCE < -3) & (bureau_balance_union.MONTHS_BALANCE >= -6) ][['SK_ID_BUREAU','SK_ID_CURR','STATUS']].groupby(['SK_ID_BUREAU','SK_ID_CURR'], as_index=False).max()
BUREAU_OVERDUE_6_0 = tmp[tmp.STATUS =='0'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_6_0'})
BUREAU_OVERDUE_6_1 = tmp[tmp.STATUS == '1'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_6_1'})
BUREAU_OVERDUE_6_2 = tmp[tmp.STATUS == '2'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_6_2'})
BUREAU_OVERDUE_6_3 = tmp[tmp.STATUS == '3'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_6_3'})
BUREAU_OVERDUE_6_4 = tmp[tmp.STATUS == '4'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_6_4'})
BUREAU_OVERDUE_6_5 = tmp[tmp.STATUS == '5'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_6_5'})

# 征信最近6-12个月逾期状态 0 1 2 3 4 5的笔数
tmp = bureau_balance_union[ (bureau_balance_union.MONTHS_BALANCE < -6) & (bureau_balance_union.MONTHS_BALANCE >= -12) ][['SK_ID_BUREAU','SK_ID_CURR','STATUS']].groupby(['SK_ID_BUREAU','SK_ID_CURR'], as_index=False).max()
BUREAU_OVERDUE_12_0 = tmp[tmp.STATUS =='0'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_12_0'})
BUREAU_OVERDUE_12_1 = tmp[tmp.STATUS == '1'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_12_1'})
BUREAU_OVERDUE_12_2 = tmp[tmp.STATUS == '2'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_12_2'})
BUREAU_OVERDUE_12_3 = tmp[tmp.STATUS == '3'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_12_3'})
BUREAU_OVERDUE_12_4 = tmp[tmp.STATUS == '4'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_12_4'})
BUREAU_OVERDUE_12_5 = tmp[tmp.STATUS == '5'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_12_5'})

# 征信最近12-24个月逾期状态 0 1 2 3 4 5的笔数
tmp = bureau_balance_union[ (bureau_balance_union.MONTHS_BALANCE < -12) & (bureau_balance_union.MONTHS_BALANCE >= -24) ][['SK_ID_BUREAU','SK_ID_CURR','STATUS']].groupby(['SK_ID_BUREAU','SK_ID_CURR'], as_index=False).max()
BUREAU_OVERDUE_24_0 = tmp[tmp.STATUS =='0'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_24_0'})
BUREAU_OVERDUE_24_1 = tmp[tmp.STATUS == '1'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_24_1'})
BUREAU_OVERDUE_24_2 = tmp[tmp.STATUS == '2'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_24_2'})
BUREAU_OVERDUE_24_3 = tmp[tmp.STATUS == '3'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_24_3'})
BUREAU_OVERDUE_24_4 = tmp[tmp.STATUS == '4'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_24_4'})
BUREAU_OVERDUE_24_5 = tmp[tmp.STATUS == '5'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_24_5'})

# 征信最近24-36个月逾期状态 0 1 2 3 4 5的笔数
tmp = bureau_balance_union[ (bureau_balance_union.MONTHS_BALANCE < -24) & (bureau_balance_union.MONTHS_BALANCE >= -36) ][['SK_ID_BUREAU','SK_ID_CURR','STATUS']].groupby(['SK_ID_BUREAU','SK_ID_CURR'], as_index=False).max()
BUREAU_OVERDUE_36_0 = tmp[tmp.STATUS =='0'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_36_0'})
BUREAU_OVERDUE_36_1 = tmp[tmp.STATUS == '1'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_36_1'})
BUREAU_OVERDUE_36_2 = tmp[tmp.STATUS == '2'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_36_2'})
BUREAU_OVERDUE_36_3 = tmp[tmp.STATUS == '3'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_36_3'})
BUREAU_OVERDUE_36_4 = tmp[tmp.STATUS == '4'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_36_4'})
BUREAU_OVERDUE_36_5 = tmp[tmp.STATUS == '5'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_36_5'})

# 征信最近36+个月逾期状态 0 1 2 3 4 5的笔数
tmp = bureau_balance_union[ bureau_balance_union.MONTHS_BALANCE < -36][['SK_ID_BUREAU','SK_ID_CURR','STATUS']].groupby(['SK_ID_BUREAU','SK_ID_CURR'], as_index=False).max()
BUREAU_OVERDUE_36plus_0 = tmp[tmp.STATUS =='0'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_36plus_0'})
BUREAU_OVERDUE_36plus_1 = tmp[tmp.STATUS == '1'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_36plus_1'})
BUREAU_OVERDUE_36plus_2 = tmp[tmp.STATUS == '2'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_36plus_2'})
BUREAU_OVERDUE_36plus_3 = tmp[tmp.STATUS == '3'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_36plus_3'})
BUREAU_OVERDUE_36plus_4 = tmp[tmp.STATUS == '4'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_36plus_4'})
BUREAU_OVERDUE_36plus_5 = tmp[tmp.STATUS == '5'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_36plus_5'})

# 删除数据
del bureau
del bureau_balance
del bureau_balance_union
del tmp

初始内存使用: 624.85 MB
优化后的内存使用: 442.60 MB


In [42]:
bureau_balance_extra = concat_df_by_name('BUREAU')
bureau_balance_extra.shape

(130774, 42)

In [43]:
# previous_application表
previous_application = load_data('previous_application')
previous_application = reduce_mem_usage(previous_application)

previous_application['RATE_INTEREST_ACTUAL'] = ((previous_application.AMT_ANNUITY * previous_application.CNT_PAYMENT) - previous_application.AMT_CREDIT) / previous_application.AMT_CREDIT
previous_application.loc[previous_application.RATE_INTEREST_ACTUAL == -1.0,'RATE_INTEREST_ACTUAL'] = np.nan
# 历史贷款总数
PRE_CREDIT_NUM = previous_application[['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_NUM'})
# 历史贷款总金额
PRE_CREDIT_AMT = previous_application[['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_AMT'})
# 历史贷款总月付款
PRE_CREDIT_ANNUITY = previous_application[['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_ANNUITY'})
# 历史刷卡贷款总数
PRE_CREDIT_POS_NUM = previous_application[previous_application['NAME_PORTFOLIO'] == 'POS'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_POS_NUM'})
# 历史刷卡贷款总金额
PRE_CREDIT_POS_AMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'POS'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_POS_AMT'})
# 历史刷卡贷款总月付款
PRE_CREDIT_POS_ANNUITY = previous_application[previous_application['NAME_PORTFOLIO'] == 'POS'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_POS_ANNUITY'})
# 历史现金贷款总数
PRE_CREDIT_CASH_NUM = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cash'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_CASH_NUM'})
# 历史现金贷款总金额
PRE_CREDIT_CASH_AMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cash'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_CASH_AMT'})
# 历史现金贷款总月付款
PRE_CREDIT_CASH_ANNUITY = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cash'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_CASH_ANNUITY'})
# 历史其他贷款总数
PRE_CREDIT_XNA_NUM = previous_application[previous_application['NAME_PORTFOLIO'] == 'XNA'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_XNA_NUM'})
# 历史其他贷款总金额
PRE_CREDIT_XNA_AMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'XNA'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_XNA_AMT'})
# 历史其他贷款总月付款
PRE_CREDIT_XNA_ANNUITY = previous_application[previous_application['NAME_PORTFOLIO'] == 'XNA'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_XNA_ANNUITY'})
# 历史信用卡贷款总数
PRE_CREDIT_Cards_NUM = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cards'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_Cards_NUM'})
# 历史信用卡贷款总金额
PRE_CREDIT_Cards_AMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cards'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_Cards_AMT'})
# 历史信用卡贷款总月付款
PRE_CREDIT_Cards_ANNUITY = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cards'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_Cards_ANNUITY'})
# 历史汽车贷款总数
PRE_CREDIT_Cars_NUM = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cars'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_Cars_NUM'})
# 历史汽车贷款总金额
PRE_CREDIT_Cars_AMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cars'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_Cars_AMT'})
# 历史汽车贷款总月付款
PRE_CREDIT_Cars_ANNUITY = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cars'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_Cars_ANNUITY'})
# 历史通过贷款总数
PRE_CREDIT_Approved_NUM = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Approved'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_Approved_NUM'})
# 历史通过贷款总金额
PRE_CREDIT_Approved_AMT = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Approved'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_Approved_AMT'})
# 历史通过贷款总月付款
PRE_CREDIT_Approved_ANNUITY = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Approved'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_Approved_ANNUITY'})
# 历史取消贷款总数
PRE_CREDIT_Canceled_NUM = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Canceled'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_Canceled_NUM'})
# 历史取消贷款总金额
PRE_CREDIT_Canceled_AMT = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Canceled'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_Canceled_AMT'})
# 历史取消贷款总月付款
PRE_CREDIT_Canceled_ANNUITY = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Canceled'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_Canceled_ANNUITY'})
# 历史被拒绝贷款总数
PRE_CREDIT_Refused_NUM = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Refused'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_Refused_NUM'})
# 历史被拒绝贷款总金额
PRE_CREDIT_Refused_AMT = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Refused'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_Refused_AMT'})
# 历史被拒绝贷款总月付款
PRE_CREDIT_Refused_ANNUITY = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Refused'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_Refused_ANNUITY'})
# 历史未使用贷款总数
PRE_CREDIT_Unused_NUM = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Unused offer'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_Unused_NUM'})
# 历史未使用贷款总金额
PRE_CREDIT_Unused_AMT = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Unused offer'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_Unused_AMT'})
# 历史未使用贷款总月付款
PRE_CREDIT_Unused_ANNUITY = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Unused offer'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_Unused_ANNUITY'})
# 拒绝原因是HC的申请数
PRE_HC_Refused_NUM = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') & (previous_application['CODE_REJECT_REASON'] == 'HC')][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_HC_Refused_NUM'})
# 拒绝原因是LIMIT的申请数
PRE_LIMIT_Refused_NUM = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') & (previous_application['CODE_REJECT_REASON'] == 'LIMIT')][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_LIMIT_Refused_NUM'})
# 拒绝原因是SCO的申请数
PRE_SCO_Refused_NUM = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') & (previous_application['CODE_REJECT_REASON'] == 'SCO')][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_SCO_Refused_NUM'})
# 拒绝原因是SCOFR的申请数
PRE_SCOFR_Refused_NUM = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') & (previous_application['CODE_REJECT_REASON'] == 'SCOFR')][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_SCOFR_Refused_NUM'})
# 拒绝原因是XNA的申请数
PRE_XNA_Refused_NUM = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') & (previous_application['CODE_REJECT_REASON'] == 'XNA')][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_XNA_Refused_NUM'})
# 拒绝原因是VERIF的申请数
PRE_VERIF_Refused_NUM = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') & (previous_application['CODE_REJECT_REASON'] == 'VERIF')][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_VERIF_Refused_NUM'})
# 拒绝原因是SYSTEM的申请数
PRE_SYSTEM_Refused_NUM = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') & (previous_application['CODE_REJECT_REASON'] == 'SYSTEM')][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_SYSTEM_Refused_NUM'})
# 历史申请最大利率
PRE_MAX_INTEREST_RATE = previous_application[['SK_ID_CURR','RATE_INTEREST_ACTUAL']].groupby(['SK_ID_CURR']).max().rename(columns={'RATE_INTEREST_ACTUAL':'PRE_MAX_INTEREST_RATE'})
# 历史申请最小利率
PRE_MIN_INTEREST_RATE = previous_application[['SK_ID_CURR','RATE_INTEREST_ACTUAL']].groupby(['SK_ID_CURR']).min().rename(columns={'RATE_INTEREST_ACTUAL':'PRE_MIN_INTEREST_RATE'})
# 历史申请平均利率
PRE_AVG_INTEREST_RATE = previous_application[['SK_ID_CURR','RATE_INTEREST_ACTUAL']].groupby(['SK_ID_CURR']).mean().rename(columns={'RATE_INTEREST_ACTUAL':'PRE_AVG_INTEREST_RATE'})

# 当前正在还的申请数
PRE_REPAY_NUM = previous_application[previous_application.DAYS_TERMINATION == 365243.0][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_NUM_REPAY'})
# 当前正在还的总金额
PRE_REPAY_AMT = previous_application[previous_application.DAYS_TERMINATION == 365243.0][['SK_ID_CURR','AMT_APPLICATION']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_APPLICATION':'PRE_REPAY_AMT'})
# 当前正在还的月付额
PRE_REPAY_ANNUITY = previous_application[previous_application.DAYS_TERMINATION == 365243.0][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_REPAY_ANNUITY'})

# 当前在还中属于逾期的申请数
PRE_REAPY_OVERDUR_NUM = previous_application[(previous_application.DAYS_TERMINATION == 365243.0) & (previous_application.DAYS_LAST_DUE != 365243.0)][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_NUM_REAPY_OVERDUR'})
# 当前正在还属于逾期的总金额
PRE_REAPY_OVERDUR_AMT = previous_application[(previous_application.DAYS_TERMINATION == 365243.0) & (previous_application.DAYS_LAST_DUE != 365243.0)][['SK_ID_CURR','AMT_APPLICATION']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_APPLICATION':'PRE_REAPY_OVERDUR_AMT'})
# 当前正在还属于逾期的月付额
PRE_REAPY_OVERDUR_ANNUITY = previous_application[(previous_application.DAYS_TERMINATION == 365243.0) & (previous_application.DAYS_LAST_DUE != 365243.0)][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_REAPY_OVERDUR_ANNUITY'})

# 最近3 6个月 1年 2年 3年 3年+贷款数 被拒绝数 通过数 贷款总金额 月付款额

del previous_application

初始内存使用: 471.48 MB
优化后的内存使用: 321.75 MB


In [44]:
pre_extra = concat_df_by_name('PRE')
pre_extra.shape

(338857, 46)

In [45]:
# POS_CASH_balance表
POS_CASH_balance = load_data('POS_CASH_balance')
POS_CASH_balance = reduce_mem_usage(POS_CASH_balance)

# pos总贷款数
POS_CREDIT_NUM = POS_CASH_balance[['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'POS_CREDIT_NUM'})
# pos已经还清的笔数
POS_FINISH_NUM = POS_CASH_balance[POS_CASH_balance.NAME_CONTRACT_STATUS == 'Completed'][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'POS_FINISH_NUM'})
# pos正在还的笔数
POS_REPAY_NUM = POS_CASH_balance[ (POS_CASH_balance.MONTHS_BALANCE == -1) & (POS_CASH_balance.NAME_CONTRACT_STATUS == 'Active')][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'POS_REPAY_NUM'})
# pos已经还清贷款最大逾期天数
tmp = pd.DataFrame(POS_CASH_balance[POS_CASH_balance.NAME_CONTRACT_STATUS == 'Completed']['SK_ID_PREV'].unique(), columns=['SK_ID_PREV'])
tmp = pd.merge(POS_CASH_balance, tmp, how='inner', on='SK_ID_PREV')
POS_DAYS_MAXOVERDUE_FINISH = tmp[['SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_CURR']).max().rename(columns={'SK_DPD_DEF':'POS_DAYS_MAXOVERDUE_FINISH'})
# pos已经还清贷款发生过逾期的笔数
POS_NUM_MAXOVERDUE_FINISH = tmp[tmp.SK_DPD_DEF > 0][['SK_ID_PREV','SK_ID_CURR']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'POS_NUM_MAXOVERDUE_FINISH'})
# pos正在还贷款最大逾期天数
tmp = pd.DataFrame(POS_CASH_balance[(POS_CASH_balance.MONTHS_BALANCE == -1) & (POS_CASH_balance.NAME_CONTRACT_STATUS == 'Active')]['SK_ID_PREV'].unique(), columns=['SK_ID_PREV'])
tmp = pd.merge(POS_CASH_balance, tmp, how='inner', on='SK_ID_PREV')
POS_DAYS_MAXOVERDUE_REPAY = tmp[['SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_CURR']).max().rename(columns={'SK_DPD_DEF':'POS_DAYS_MAXOVERDUE_REPAY'})
# pos正在还贷款发生过逾期的笔数
POS_NUM_MAXOVERDUE_REPAY = tmp[tmp.SK_DPD_DEF > 0][['SK_ID_PREV','SK_ID_CURR']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'POS_NUM_MAXOVERDUE_REPAY'})

# pos最近6个月逾期0 7 14 30 90 90+天内的笔数
tmp = POS_CASH_balance[(POS_CASH_balance.MONTHS_BALANCE >= -6)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
POS_OVERDUE_6_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_6_0'})
POS_OVERDUE_6_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_6_7'})
POS_OVERDUE_6_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_6_14'})
POS_OVERDUE_6_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_6_30'})
POS_OVERDUE_6_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_6_90'})
POS_OVERDUE_6_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_6_90plus'})

# pos最近7-12个月逾期0 7 14 30 90 90+天内的笔数
tmp = POS_CASH_balance[(POS_CASH_balance.MONTHS_BALANCE >= -12) & (POS_CASH_balance.MONTHS_BALANCE < -6)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
POS_OVERDUE_12_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_12_0'})
POS_OVERDUE_12_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_12_7'})
POS_OVERDUE_12_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_12_14'})
POS_OVERDUE_12_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_12_30'})
POS_OVERDUE_12_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_12_90'})
POS_OVERDUE_12_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_12_90plus'})

# pos最近13-24个月逾期0 7 14 30 90 90+天内的笔数
tmp = POS_CASH_balance[(POS_CASH_balance.MONTHS_BALANCE >= -24) & (POS_CASH_balance.MONTHS_BALANCE < -12)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
POS_OVERDUE_24_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_24_0'})
POS_OVERDUE_24_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_24_7'})
POS_OVERDUE_24_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_24_14'})
POS_OVERDUE_24_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_24_30'})
POS_OVERDUE_24_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_24_90'})
POS_OVERDUE_24_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_24_90plus'})

# pos最近24-36个月逾期0 7 14 30 90 90+天内的笔数
tmp = POS_CASH_balance[(POS_CASH_balance.MONTHS_BALANCE >= -36) & (POS_CASH_balance.MONTHS_BALANCE < -24)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
POS_OVERDUE_36_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_36_0'})
POS_OVERDUE_36_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_36_7'})
POS_OVERDUE_36_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_36_14'})
POS_OVERDUE_36_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_36_30'})
POS_OVERDUE_36_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_36_90'})
POS_OVERDUE_36_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_36_90plus'})

# pos最近36以上个月逾期0 7 14 30 90 90+天内的笔数
tmp = POS_CASH_balance[POS_CASH_balance.MONTHS_BALANCE < -36][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
POS_OVERDUE_36plus_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_36plus_0'})
POS_OVERDUE_36plus_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_36plus_7'})
POS_OVERDUE_36plus_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_36plus_14'})
POS_OVERDUE_36plus_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_36plus_30'})
POS_OVERDUE_36plus_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_36plus_90'})
POS_OVERDUE_36plus_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUE_36plus_90plus'})

# pos当前仍在逾期的笔数 最大逾期天数
POS_NUM_OVERDUE_STILL = POS_CASH_balance[(POS_CASH_balance.MONTHS_BALANCE == -1) & (POS_CASH_balance.SK_DPD_DEF > 30)][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_NUM_OVERDUE_STILL'})
POS_DAYS_MAXOVERDUE_STILL = POS_CASH_balance[(POS_CASH_balance.MONTHS_BALANCE == -1) & (POS_CASH_balance.SK_DPD_DEF > 30)][['SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_CURR']).max().rename(columns={'SK_DPD_DEF':'POS_DAYS_MAXOVERDUE_STILL'})

del POS_CASH_balance
del tmp

初始内存使用: 610.43 MB
优化后的内存使用: 314.76 MB


In [46]:
pos_extra = concat_df_by_name('POS')
pos_extra.shape

(337252, 39)

In [47]:
# installments_payments表
installments_payments = load_data('installments_payments')
installments_payments = reduce_mem_usage(installments_payments)

# 分期还款记录中有很多应还款额为67.5的记录，中间有大量的逾期，明显有问题，我们剔除这些数据
installments_payments = installments_payments[installments_payments.AMT_INSTALMENT > 100]
installments_payments['DAYS_DIFF'] = installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']
# 分期还款的贷款数
INST_NUM = installments_payments[['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM'})
# 分期还款最近6个月在还贷款数
INST_NUM_6m = installments_payments[installments_payments.DAYS_INSTALMENT >= -180][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_6m'})
# 分期还款最近6个月发生了逾期的贷款数
INST_NUM_6m_all = installments_payments[(installments_payments.DAYS_INSTALMENT >= -180) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_6m_all'})
# 分期还款最近6个月发生了逾期，逾期天数在7天内的贷款数
INST_NUM_6m_7d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -180) & (installments_payments.DAYS_DIFF > 0) & (installments_payments.DAYS_DIFF <= 7)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_6m_7d'})
# 分期还款最近6个月发生了逾期，逾期天数在8-14天内的贷款数
INST_NUM_6m_14d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -180) & (installments_payments.DAYS_DIFF > 7) & (installments_payments.DAYS_DIFF <= 14)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_6m_14d'})
# 分期还款最近6个月发生了逾期，逾期天数在15-30天内的贷款数
INST_NUM_6m_30d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -180) & (installments_payments.DAYS_DIFF > 14) & (installments_payments.DAYS_DIFF <= 30)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_6m_30d'})
# 分期还款最近6个月发生了逾期，逾期天数在31-90天内的贷款数
INST_NUM_6m_90d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -180) & (installments_payments.DAYS_DIFF > 30) & (installments_payments.DAYS_DIFF <= 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_6m_90d'})
# 分期还款最近6个月发生了逾期，逾期天数在90+天内的贷款数
INST_NUM_6m_90dplus = installments_payments[(installments_payments.DAYS_INSTALMENT >= -180) & (installments_payments.DAYS_DIFF > 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_6m_90dplus'})

# 分期还款最近6-12个月在还贷款数
INST_NUM_12m = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_12m'})
# 分期还款最近6-12个月发生了逾期的贷款数
INST_NUM_12m_all = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_12m_all'})
# 分期还款最近6-12个月发生了逾期，逾期天数在7天内的贷款数
INST_NUM_12m_7d = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360) & (installments_payments.DAYS_DIFF > 0) & (installments_payments.DAYS_DIFF <= 7)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_12m_7d'})
# 分期还款最近6-12个月发生了逾期，逾期天数在8-14天内的贷款数
INST_NUM_12m_14d = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360) & (installments_payments.DAYS_DIFF > 7) & (installments_payments.DAYS_DIFF <= 14)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_12m_14d'})
# 分期还款最近6-12个月发生了逾期，逾期天数在15-30天内的贷款数
INST_NUM_12m_30d = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360) & (installments_payments.DAYS_DIFF > 14) & (installments_payments.DAYS_DIFF <= 30)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_12m_30d'})
# 分期还款最近6-12个月发生了逾期，逾期天数在31-90天内的贷款数
INST_NUM_12m_90d = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360) & (installments_payments.DAYS_DIFF > 30) & (installments_payments.DAYS_DIFF <= 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_12m_90d'})
# 分期还款最近6-12个月发生了逾期，逾期天数在90+天内的贷款数
INST_NUM_12m_90dplus = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360) & (installments_payments.DAYS_DIFF > 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_12m_90dplus'})

# 分期还款最近12-24个月在还贷款数
INST_NUM_24m = installments_payments[(installments_payments.DAYS_INSTALMENT <= -720) & (installments_payments.DAYS_INSTALMENT < -360)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_24m'})
# 分期还款最近12-24个月发生了逾期的贷款数
INST_NUM_24m_all = installments_payments[(installments_payments.DAYS_INSTALMENT <= -720) & (installments_payments.DAYS_INSTALMENT < -360) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_24m_all'})
# 分期还款最近12-24个月发生了逾期，逾期天数在7天内的贷款数
INST_NUM_24m_7d = installments_payments[(installments_payments.DAYS_INSTALMENT <= -720) & (installments_payments.DAYS_INSTALMENT < -360) & (installments_payments.DAYS_DIFF > 0) & (installments_payments.DAYS_DIFF <= 7)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_24m_7d'})
# 分期还款最近12-24个月发生了逾期，逾期天数在8-14天内的贷款数
INST_NUM_24m_14d = installments_payments[(installments_payments.DAYS_INSTALMENT <= -720) & (installments_payments.DAYS_INSTALMENT < -360) & (installments_payments.DAYS_DIFF > 7) & (installments_payments.DAYS_DIFF <= 14)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_24m_14d'})
# 分期还款最近12-24个月发生了逾期，逾期天数在15-30天内的贷款数
INST_NUM_24m_30d = installments_payments[(installments_payments.DAYS_INSTALMENT <= -720) & (installments_payments.DAYS_INSTALMENT < -360) & (installments_payments.DAYS_DIFF > 14) & (installments_payments.DAYS_DIFF <= 30)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_24m_30d'})
# 分期还款最近12-24个月发生了逾期，逾期天数在31-90天内的贷款数
INST_NUM_24m_90d = installments_payments[(installments_payments.DAYS_INSTALMENT <= -720) & (installments_payments.DAYS_INSTALMENT < -360) & (installments_payments.DAYS_DIFF > 30) & (installments_payments.DAYS_DIFF <= 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_24m_90d'})
# 分期还款最近12-24个月发生了逾期，逾期天数在90+天内的贷款数
INST_NUM_24m_90dplus = installments_payments[(installments_payments.DAYS_INSTALMENT <= -720) & (installments_payments.DAYS_INSTALMENT < -360) & (installments_payments.DAYS_DIFF > 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_24m_90dplus'})

# 分期还款最近24-36个月在还贷款数
INST_NUM_36m = installments_payments[(installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_INSTALMENT < -720)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36m'})
# 分期还款最近24-36个月发生了逾期的贷款数
INST_NUM_36m_all = installments_payments[(installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_INSTALMENT < -720) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36m_all'})
# 分期还款最近24-36个月发生了逾期，逾期天数在7天内的贷款数
INST_NUM_36m_7d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_INSTALMENT < -720) & (installments_payments.DAYS_DIFF > 0) & (installments_payments.DAYS_DIFF <= 7)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36m_7d'})
# 分期还款最近24-36个月发生了逾期，逾期天数在8-14天内的贷款数
INST_NUM_36m_14d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_INSTALMENT < -720) & (installments_payments.DAYS_DIFF > 7) & (installments_payments.DAYS_DIFF <= 14)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36m_14d'})
# 分期还款最近24-36个月发生了逾期，逾期天数在15-30天内的贷款数
INST_NUM_36m_30d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_INSTALMENT < -720) & (installments_payments.DAYS_DIFF > 14) & (installments_payments.DAYS_DIFF <= 30)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36m_30d'})
# 分期还款最近24-36个月发生了逾期，逾期天数在31-90天内的贷款数
INST_NUM_36m_90d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_INSTALMENT < -720) & (installments_payments.DAYS_DIFF > 30) & (installments_payments.DAYS_DIFF <= 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36m_90d'})
# 分期还款最近24-36个月发生了逾期，逾期天数在90+天内的贷款数
INST_NUM_36m_90dplus = installments_payments[(installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_INSTALMENT < -720) & (installments_payments.DAYS_DIFF > 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36m_90dplus'})

# 分期还款最近36+个月在还贷款数
INST_NUM_36mplus = installments_payments[installments_payments.DAYS_INSTALMENT < -1080][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36mplus'})
# 分期还款最近36+个月发生了逾期的贷款数
INST_NUM_36mplus_all = installments_payments[installments_payments.DAYS_INSTALMENT < -1080 & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36mplus_all'})
# 分期还款最近36+个月发生了逾期，逾期天数在7天内的贷款数
INST_NUM_36mplus_7d = installments_payments[installments_payments.DAYS_INSTALMENT < -1080 & (installments_payments.DAYS_DIFF > 0) & (installments_payments.DAYS_DIFF <= 7)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36mplus_7d'})
# 分期还款最近36+个月发生了逾期，逾期天数在8-14天内的贷款数
INST_NUM_36mplus_14d = installments_payments[installments_payments.DAYS_INSTALMENT < -1080 & (installments_payments.DAYS_DIFF > 7) & (installments_payments.DAYS_DIFF <= 14)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36mplus_14d'})
# 分期还款最近36+个月发生了逾期，逾期天数在15-30天内的贷款数
INST_NUM_36mplus_30d = installments_payments[installments_payments.DAYS_INSTALMENT < -1080 & (installments_payments.DAYS_DIFF > 14) & (installments_payments.DAYS_DIFF <= 30)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36mplus_30d'})
# 分期还款最近36+个月发生了逾期，逾期天数在31-90天内的贷款数
INST_NUM_36mplus_90d = installments_payments[installments_payments.DAYS_INSTALMENT < -1080 & (installments_payments.DAYS_DIFF > 30) & (installments_payments.DAYS_DIFF <= 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36mplus_90d'})
# 分期还款最近36+个月发生了逾期，逾期天数在90+天内的贷款数
INST_NUM_36mplus_90dplus = installments_payments[installments_payments.DAYS_INSTALMENT < -1080 & (installments_payments.DAYS_DIFF > 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36mplus_90dplus'})

# 分期还款最近6个月发生逾期的逾期金额
INST_AMT_6m = installments_payments[(installments_payments.DAYS_INSTALMENT >= -180) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','AMT_PAYMENT']].groupby('SK_ID_CURR').sum().rename(columns={'AMT_PAYMENT':'INST_AMT_6m'})
# 分期还款最近6-12个月发生逾期的逾期金额
INST_AMT_12m = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','AMT_PAYMENT']].groupby('SK_ID_CURR').sum().rename(columns={'AMT_PAYMENT':'INST_AMT_12m'})
# 分期还款最近12-24个月发生逾期的逾期金额
INST_AMT_24m = installments_payments[(installments_payments.DAYS_INSTALMENT < -360) & (installments_payments.DAYS_INSTALMENT >= -720) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','AMT_PAYMENT']].groupby('SK_ID_CURR').sum().rename(columns={'AMT_PAYMENT':'INST_AMT_24m'})
# 分期还款最近24-36个月发生逾期的逾期金额
INST_AMT_36m = installments_payments[(installments_payments.DAYS_INSTALMENT < -720) & (installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','AMT_PAYMENT']].groupby('SK_ID_CURR').sum().rename(columns={'AMT_PAYMENT':'INST_AMT_36m'})
# 分期还款最近36+个月发生逾期的逾期金额
INST_AMT_36mplus = installments_payments[(installments_payments.DAYS_INSTALMENT < -1080) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','AMT_PAYMENT']].groupby('SK_ID_CURR').sum().rename(columns={'AMT_PAYMENT':'INST_AMT_36mplus'})

# 分期还款当前仍在逾期的贷款数
INST_NUM_STILL = installments_payments[installments_payments.DAYS_ENTRY_PAYMENT.isnull()][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'INST_NUM_STILL'})
# 分期还款当前仍在逾期的贷款总逾期期数
INST_NUM_SEQ_STILL = installments_payments[installments_payments.DAYS_ENTRY_PAYMENT.isnull()][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'INST_NUM_SEQ_STILL'})
# 分期还款当前仍在逾期的贷款最大逾期天数
INST_DAYS_MAX_STILL = installments_payments[installments_payments.DAYS_ENTRY_PAYMENT.isnull()][['SK_ID_CURR','DAYS_INSTALMENT']].groupby(['SK_ID_CURR']).min().rename(columns={'DAYS_INSTALMENT':'INST_DAYS_MAX_STILL'})
# 分期还款当前仍在逾期的总逾期金额
INST_AMT_STILL = installments_payments[installments_payments.DAYS_ENTRY_PAYMENT.isnull()][['SK_ID_CURR','AMT_INSTALMENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_INSTALMENT':'INST_AMT_STILL'})

del installments_payments

初始内存使用: 830.41 MB
优化后的内存使用: 415.20 MB


In [48]:
inst_extra = concat_df_by_name('INST')
inst_extra.shape

(339572, 45)

In [49]:
# credit_card_balance表
credit_card_balance = load_data('credit_card_balance')
credit_card_balance = reduce_mem_usage(credit_card_balance)

# 信用卡数
CREDIT_NUM = credit_card_balance[['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'CREDIT_NUM'})
# 不同状态的信用卡数
CREDIT_NUM_ACTIVE = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Active'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'CREDIT_NUM_ACTIVE'})
CREDIT_NUM_COMPLETED = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Completed'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'CREDIT_NUM_COMPLETED'})
CREDIT_NUM_SIGNED = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Signed'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'CREDIT_NUM_SIGNED'})
CREDIT_NUM_DEMAND = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Demand'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'CREDIT_NUM_DEMAND'})
CREDIT_NUM_SENT = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Sent proposal'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'CREDIT_NUM_SENT'})
CREDIT_NUM_REFUSED = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Refused'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'CREDIT_NUM_REFUSED'})
# 信用卡使用时长
tmp = credit_card_balance[['SK_ID_PREV','SK_ID_CURR','MONTHS_BALANCE']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).count()
CREDIT_MONTHS_MAX = tmp[['SK_ID_CURR','MONTHS_BALANCE']].groupby('SK_ID_CURR').max().rename(columns={'MONTHS_BALANCE':'CREDIT_MONTHS_MAX'})
CREDIT_MONTHS_MIN = tmp[['SK_ID_CURR','MONTHS_BALANCE']].groupby('SK_ID_CURR').min().rename(columns={'MONTHS_BALANCE':'CREDIT_MONTHS_MIN'})
CREDIT_MONTHS_AVG = tmp[['SK_ID_CURR','MONTHS_BALANCE']].groupby('SK_ID_CURR').mean().rename(columns={'MONTHS_BALANCE':'CREDIT_MONTHS_AVG'})
CREDIT_MONTHS_SUM = tmp[['SK_ID_CURR','MONTHS_BALANCE']].groupby('SK_ID_CURR').sum().rename(columns={'MONTHS_BALANCE':'CREDIT_MONTHS_SUM'})
# 信用卡最近1 3 6 12 24 36 36+个月 月均余额
CREDIT_AMT_1m = credit_card_balance[credit_card_balance.MONTHS_BALANCE == -1][['SK_ID_CURR','AMT_BALANCE']].groupby(['SK_ID_CURR']).mean().rename(columns={'AMT_BALANCE':'CREDIT_AMT_1m'})
CREDIT_AMT_3m = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -3][['SK_ID_CURR','AMT_BALANCE']].groupby(['SK_ID_CURR']).mean().rename(columns={'AMT_BALANCE':'CREDIT_AMT_3m'})
CREDIT_AMT_6m = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -6][['SK_ID_CURR','AMT_BALANCE']].groupby(['SK_ID_CURR']).mean().rename(columns={'AMT_BALANCE':'CREDIT_AMT_6m'})
CREDIT_AMT_12m = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -12][['SK_ID_CURR','AMT_BALANCE']].groupby(['SK_ID_CURR']).mean().rename(columns={'AMT_BALANCE':'CREDIT_AMT_12m'})
CREDIT_AMT_24m = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -24][['SK_ID_CURR','AMT_BALANCE']].groupby(['SK_ID_CURR']).mean().rename(columns={'AMT_BALANCE':'CREDIT_AMT_24m'})
CREDIT_AMT_36m = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -36][['SK_ID_CURR','AMT_BALANCE']].groupby(['SK_ID_CURR']).mean().rename(columns={'AMT_BALANCE':'CREDIT_AMT_36m'})
CREDIT_AMT_36mplus = credit_card_balance[credit_card_balance.MONTHS_BALANCE < -36][['SK_ID_CURR','AMT_BALANCE']].groupby(['SK_ID_CURR']).mean().rename(columns={'AMT_BALANCE':'CREDIT_AMT_36mplus'})
# 信用卡最近1 3 6 12 24 36 36+个月取款金额
CREDIT_AMT_1m_CURRENT = credit_card_balance[credit_card_balance.MONTHS_BALANCE == -1][['SK_ID_CURR','AMT_DRAWINGS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_CURRENT':'CREDIT_AMT_1m_CURRENT'})
CREDIT_AMT_3m_CURRENT = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -3][['SK_ID_CURR','AMT_DRAWINGS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_CURRENT':'CREDIT_AMT_3m_CURRENT'})
CREDIT_AMT_6m_CURRENT = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -6][['SK_ID_CURR','AMT_DRAWINGS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_CURRENT':'CREDIT_AMT_6m_CURRENT'})
CREDIT_AMT_12m_CURRENT = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -12][['SK_ID_CURR','AMT_DRAWINGS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_CURRENT':'CREDIT_AMT_12m_CURRENT'})
CREDIT_AMT_24m_CURRENT = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -24][['SK_ID_CURR','AMT_DRAWINGS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_CURRENT':'CREDIT_AMT_24m_CURRENT'})
CREDIT_AMT_36m_CURRENT = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -36][['SK_ID_CURR','AMT_DRAWINGS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_CURRENT':'CREDIT_AMT_36m_CURRENT'})
CREDIT_AMT_36mplus_CURRENT = credit_card_balance[credit_card_balance.MONTHS_BALANCE < -36][['SK_ID_CURR','AMT_DRAWINGS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_CURRENT':'CREDIT_AMT_36mplus_CURRENT'})
# 信用卡最近1 3 6 12 24 36 36+个月POS金额
CREDIT_AMT_1m_POS = credit_card_balance[credit_card_balance.MONTHS_BALANCE == -1][['SK_ID_CURR','AMT_DRAWINGS_POS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_POS_CURRENT':'CREDIT_AMT_1m_POS'})
CREDIT_AMT_3m_POS = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -3][['SK_ID_CURR','AMT_DRAWINGS_POS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_POS_CURRENT':'CREDIT_AMT_3m_POS'})
CREDIT_AMT_6m_POS = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -6][['SK_ID_CURR','AMT_DRAWINGS_POS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_POS_CURRENT':'CREDIT_AMT_6m_POS'})
CREDIT_AMT_12m_POS = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -12][['SK_ID_CURR','AMT_DRAWINGS_POS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_POS_CURRENT':'CREDIT_AMT_12m_POS'})
CREDIT_AMT_24m_POS = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -24][['SK_ID_CURR','AMT_DRAWINGS_POS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_POS_CURRENT':'CREDIT_AMT_24m_POS'})
CREDIT_AMT_36m_POS = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -36][['SK_ID_CURR','AMT_DRAWINGS_POS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_POS_CURRENT':'CREDIT_AMT_36m_POS'})
CREDIT_AMT_36mplus_POS = credit_card_balance[credit_card_balance.MONTHS_BALANCE < -36][['SK_ID_CURR','AMT_DRAWINGS_POS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_POS_CURRENT':'CREDIT_AMT_36mplus_POS'})
# 信用卡最近1 3 6 12 24 36 36+个月ATM金额
CREDIT_AMT_1m_ATM = credit_card_balance[credit_card_balance.MONTHS_BALANCE == -1][['SK_ID_CURR','AMT_DRAWINGS_ATM_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_ATM_CURRENT':'CREDIT_AMT_1m_ATM'})
CREDIT_AMT_3m_ATM = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -3][['SK_ID_CURR','AMT_DRAWINGS_ATM_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_ATM_CURRENT':'CREDIT_AMT_3m_ATM'})
CREDIT_AMT_6m_ATM = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -6][['SK_ID_CURR','AMT_DRAWINGS_ATM_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_ATM_CURRENT':'CREDIT_AMT_6m_ATM'})
CREDIT_AMT_12m_ATM = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -12][['SK_ID_CURR','AMT_DRAWINGS_ATM_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_ATM_CURRENT':'CREDIT_AMT_12m_ATM'})
CREDIT_AMT_24m_ATM = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -24][['SK_ID_CURR','AMT_DRAWINGS_ATM_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_ATM_CURRENT':'CREDIT_AMT_24m_ATM'})
CREDIT_AMT_36m_ATM = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -36][['SK_ID_CURR','AMT_DRAWINGS_ATM_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_ATM_CURRENT':'CREDIT_AMT_36m_ATM'})
CREDIT_AMT_36mplus_ATM = credit_card_balance[credit_card_balance.MONTHS_BALANCE < -36][['SK_ID_CURR','AMT_DRAWINGS_ATM_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_ATM_CURRENT':'CREDIT_AMT_36mplus_ATM'})
# 信用卡最大 最小 平均额度
CREDIT_LIMIT_MAX = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Active'][['SK_ID_CURR','AMT_CREDIT_LIMIT_ACTUAL']].groupby(['SK_ID_CURR']).max().rename(columns={'AMT_CREDIT_LIMIT_ACTUAL':'CREDIT_LIMIT_MAX'})
CREDIT_LIMIT_MIN = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Active'][['SK_ID_CURR','AMT_CREDIT_LIMIT_ACTUAL']].groupby(['SK_ID_CURR']).min().rename(columns={'AMT_CREDIT_LIMIT_ACTUAL':'CREDIT_LIMIT_MIN'})
# 信用卡最近6个月逾期0 7 14 30 90 90+天内的笔数
tmp = credit_card_balance[(credit_card_balance.MONTHS_BALANCE >= -6)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
CREDIT_OVERDUE_6_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_6_0'})
CREDIT_OVERDUE_6_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_6_7'})
CREDIT_OVERDUE_6_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_6_14'})
CREDIT_OVERDUE_6_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_6_30'})
CREDIT_OVERDUE_6_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_6_90'})
CREDIT_OVERDUE_6_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_6_90plus'})

# 信用卡最近7-12个月逾期0 7 14 30 90 90+天内的笔数
tmp = credit_card_balance[(credit_card_balance.MONTHS_BALANCE >= -12) & (credit_card_balance.MONTHS_BALANCE < -6)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
CREDIT_OVERDUE_12_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_12_0'})
CREDIT_OVERDUE_12_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_12_7'})
CREDIT_OVERDUE_12_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_12_14'})
CREDIT_OVERDUE_12_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_12_30'})
CREDIT_OVERDUE_12_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_12_90'})
CREDIT_OVERDUE_12_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_12_90plus'})

# 信用卡最近13-24个月逾期0 7 14 30 90 90+天内的笔数
tmp = credit_card_balance[(credit_card_balance.MONTHS_BALANCE >= -24) & (credit_card_balance.MONTHS_BALANCE < -12)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
CREDIT_OVERDUE_24_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_24_0'})
CREDIT_OVERDUE_24_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_24_7'})
CREDIT_OVERDUE_24_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_24_14'})
CREDIT_OVERDUE_24_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_24_30'})
CREDIT_OVERDUE_24_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_24_90'})
CREDIT_OVERDUE_24_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_24_90plus'})

# 信用卡最近24-36个月逾期0 7 14 30 90 90+天内的笔数
tmp = credit_card_balance[(credit_card_balance.MONTHS_BALANCE >= -36) & (credit_card_balance.MONTHS_BALANCE < -24)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
CREDIT_OVERDUE_36_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_36_0'})
CREDIT_OVERDUE_36_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_36_7'})
CREDIT_OVERDUE_36_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_36_14'})
CREDIT_OVERDUE_36_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_36_30'})
CREDIT_OVERDUE_36_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_36_90'})
CREDIT_OVERDUE_36_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_36_90plus'})

# 信用卡最近36以上个月逾期0 7 14 30 90 90+天内的笔数
tmp = credit_card_balance[credit_card_balance.MONTHS_BALANCE < -36][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
CREDIT_OVERDUE_36plus_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_36plus_0'})
CREDIT_OVERDUE_36plus_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_36plus_7'})
CREDIT_OVERDUE_36plus_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_36plus_14'})
CREDIT_OVERDUE_36plus_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_36plus_30'})
CREDIT_OVERDUE_36plus_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_36plus_90'})
CREDIT_OVERDUE_36plus_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUE_36plus_90plus'})

# 信用卡当前仍在逾期的笔数 最大逾期天数
# CREDIT_NUM_OVERDUE_STILL = credit_card_balance[(credit_card_balance.MONTHS_BALANCE == -1) & (credit_card_balance.SK_DPD_DEF > 30)][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'CREDIT_NUM_OVERDUE_STILL'})
# CREDIT_DAYS_MAXOVERDUE_STILL = credit_card_balance[(credit_card_balance.MONTHS_BALANCE == -1) & (credit_card_balance.SK_DPD_DEF > 30)][['SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_CURR']).max().rename(columns={'SK_DPD_DEF':'CREDIT_DAYS_MAXOVERDUE_STILL'})

del credit_card_balance
del tmp

初始内存使用: 673.88 MB
优化后的内存使用: 318.63 MB


In [50]:
credit_extra = concat_df_by_name('CREDIT')
credit_extra.shape

(103558, 71)

In [51]:
extra_info = [ bureau_extra, bureau_balance_extra, pre_extra, pos_extra, inst_extra, credit_extra ]

In [52]:
def merge_info(df,ls):
    print('合并前shape:{}'.format(df.shape))
    res = df.set_index('SK_ID_CURR')
    for extra in ls:
        res = pd.merge(res, extra, how = 'left', left_index=True, right_index=True)
    print('合并前shape:{}'.format(res.shape))
    return res

In [53]:
app = train_set.copy()
app_labels = train_set['TARGET'].copy()

In [54]:
app_extra = merge_info(app, extra_info)

合并前shape:(246008, 122)
合并前shape:(246008, 407)


In [55]:
missing_values_summary(app_extra)

Your selected dataframe has 407 columns.
There are 353 columns that have missing values.


,Missing Values,% of Total Values,dtype
CREDIT_OVERDUE_24_90,246007,100.0,float64
CREDIT_OVERDUE_12_90,246006,100.0,float64
POS_OVERDUE_24_90,246005,100.0,float64
CREDIT_OVERDUE_6_90,246003,100.0,float64
CREDIT_OVERDUE_12_90plus,246001,100.0,float64
CREDIT_OVERDUE_24_90plus,246001,100.0,float64
POS_OVERDUE_12_90,246000,100.0,float64
CREDIT_OVERDUE_36_90plus,246000,100.0,float64
CREDIT_OVERDUE_6_30,245999,100.0,float64
CREDIT_NUM_DEMAND,245997,100.0,float64


In [82]:
app_extra.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BUREAU_NUM,BUREAU_ACTIVE_NUM,BUREAU_Closed_NUM,BUREAU_SOLD_NUM,BUREAU_BAD_DEBT_NUM,BUREAU_MINDAY_APPLICATION,BUREAU_MAXDAY_APPLICATION,BUREAU_NUM_OVERDUE,BUREAU_MAXDAY_OVERDUE,BUREAU_NUM_PREPAY,BUREAU_NUM_NORMAL,BUREAU_NUM_DELAY,BUREAU_MAXAMT_OVERDUE,BUREAU_PROLONG_NUM,BUREAU_LOAN_AMT,BUREAU_DEBT_AMT,BUREAU_LIMIT_MAX,BUREAU_SUM_OVERDUE,BUREAU_NUM_CONSUMER,BUREAU_NUM_CARD,BUREAU_NUM_CAR,BUREAU_NUM_MORTGAGE,BUREAU_NUM_MiCROLOAN,BUREAU_NUM_OTHER,BUREAU_AMT_CONSUMER,BUREAU_AMT_CARD,BUREAU_AMT_CAR,BUREAU_AMT_MORTGAGE,BUREAU_AMT_MiCROLOAN,BUREAU_AMT_OTHER,BUREAU_DEBTAMT_CONSUMER,BUREAU_DEBTAMT_CARD,BUREAU_DEBTAMT_CAR,BUREAU_DEBTAMT_MORTGAGE,BUREAU_DEBTAMT_MiCROLOAN,BUREAU_DEBTAMT_OTHER,BUREAU_OVERDUEAMT_CONSUMER,BUREAU_OVERDUEAMT_CARD,BUREAU_OVERDUEAMT_CAR,BUREAU_OVERDUEAMT_MORTGAGE,BUREAU_OVERDUEAMT_MiCROLOAN,BUREAU_OVERDUEAMT_OTHER,BUREAU_LAST_UPDATE,BUREAU_OVERDUE_1_0,BUREAU_OVERDUE_1_1,BUREAU_OVERDUE_1_2,BUREAU_OVERDUE_1_3,BUREAU_OVERDUE_1_4,BUREAU_OVERDUE_1_5,BUREAU_OVERDUE_3_0,BUREAU_OVERDUE_3_1,BUREAU_OVERDUE_3_2,BUREAU_OVERDUE_3_3,BUREAU_OVERDUE_3_4,BUREAU_OVERDUE_3_5,BUREAU_OVERDUE_6_0,BUREAU_OVERDUE_6_1,BUREAU_OVERDUE_6_2,BUREAU_OVERDUE_6_3,BUREAU_OVERDUE_6_4,BUREAU_OVERDUE_6_5,BUREAU_OVERDUE_12_0,BUREAU_OVERDUE_12_1,BUREAU_OVERDUE_12_2,BUREAU_OVERDUE_12_3,BUREAU_OVERDUE_12_4,BUREAU_OVERDUE_12_5,BUREAU_OVERDUE_24_0,BUREAU_OVERDUE_24_1,BUREAU_OVERDUE_24_2,BUREAU_OVERDUE_24_3,BUREAU_OVERDUE_24_4,BUREAU_OVERDUE_24_5,BUREAU_OVERDUE_36_0,BUREAU_OVERDUE_36_1,BUREAU_OVERDUE_36_2,BUREAU_OVERDUE_36_3,BUREAU_OVERDUE_36_4,BUREAU_OVERDUE_36_5,BUREAU_OVERDUE_36plus_0,BUREAU_OVERDUE_36plus_1,BUREAU_OVERDUE_36plus_2,BUREAU_OVERDUE_36plus_3,BUREAU_OVERDUE_36plus_4,BUREAU_OVERDUE_36plus_5,PRE_CREDIT_NUM,PRE_CREDIT_AMT,PRE_CREDIT_ANNUITY,PRE_CREDIT_POS_NUM,PRE_CREDIT_POS_AMT,PRE_CREDIT_POS_AN